In [1]:
import random
import numpy as np
import pandas as pd
import time
import re
import datetime
import os
# import matplotlib.pyplot as plt
# import gensim
# import spacy

In [2]:
import spacy
# pip install -U spacy
# !python -m spacy download ru_core_news_md
nlp = spacy.load('ru_core_news_lg')

In [3]:
df = pd.read_excel('Excel_files/Full_Poem_Dataset_9-18_0.xlsx')
df = df.drop('Unnamed: 0',axis=1)
records = df.to_dict('records')

print(len(df))
df.head()

2158


,Text,Author,Before or after,Source
0,Стихи о постсоветском человеке\n\n1.\nДесять з...,Александр Бараш,After,ROAR
1,2.\nПо капле выдавливать из себя \nсоветского ...,Александр Бараш,After,ROAR
2,"Дети\n\nСто шесть человек прибыли, тридцать че...",Александр Ланин,After,ROAR
3,Развод\n\nОднажды моя любимая дала пощёчину ре...,Александр Ланин,After,ROAR
4,Круциатус\n\nНевилл Лонгботтом стоит и смотрит...,Александр Ланин,After,ROAR


## Run Spacy

In [4]:
%%time
badIdxs = []
for i, rec in enumerate(records):
    if i % 100 == 0:
        print(f'{i}/{len(records)} parsed by Spacy.')
    
    try:
        rec['doc'] = nlp(rec['Text'])
    except:
        badIdxs.append(i)
for i in badIdxs:
    records.pop(i)

0/2158 parsed by Spacy.
100/2158 parsed by Spacy.
200/2158 parsed by Spacy.
300/2158 parsed by Spacy.
400/2158 parsed by Spacy.
500/2158 parsed by Spacy.
600/2158 parsed by Spacy.
700/2158 parsed by Spacy.
800/2158 parsed by Spacy.
900/2158 parsed by Spacy.
1000/2158 parsed by Spacy.
1100/2158 parsed by Spacy.
1200/2158 parsed by Spacy.
1300/2158 parsed by Spacy.
1400/2158 parsed by Spacy.
1500/2158 parsed by Spacy.
1600/2158 parsed by Spacy.
1700/2158 parsed by Spacy.
1800/2158 parsed by Spacy.
1900/2158 parsed by Spacy.
2000/2158 parsed by Spacy.
2100/2158 parsed by Spacy.
CPU times: user 4min 9s, sys: 17.4 s, total: 4min 27s
Wall time: 4min 39s


## Text cleaning

In [124]:

def initializeRec(recText):
    if isinstance(recText, str):
        return recText.strip()
    else:
        return False

def skipLine(line, idx):
    if len(line.strip()) == 0:
        return False
    
    # throw hashtag line
    if line.strip()[0] == '#':
        return True
    
    # throw attribution line
    for attr in ['из личного','личный блог','источник:','авторский блог']:
        if attr in line.lower():
            return True
        
    # dots at the beginning
    matches = re.search("[\*\+\^-_][*+^-_= ]+", line.strip()) 
    if matches:
        return True
    
    # is none of it alphanumeric
    containsAlpha = False
    for char in line:
        if char.isalpha():
            containsAlpha = True
            break
    if not containsAlpha:
        return True
    
    return False


def processRec(rec):
    recText = initializeRec(rec['Text'])
    cleanLines = []
    if recText:
        # decide which lines to keep
        lines = recText.split('\n')
        for i, line in enumerate(lines):
            if skipLine(line, i):
                continue
            cleanLines.append(line)
    return '\n'.join(cleanLines)

In [128]:
# r = random.choice(records)
# print(r['Text'])
# print()
# print(processRec(r))

## Testing out the NLP capabilities

In [5]:

rec = random.choice(records)
txt = rec['Text']
# for txt in lines:
doc = nlp(txt)

NERcounter = dict()
for ent in doc.ents:
    lemma = ent.lemma_
    entType = ent.label_
    NERcounter.setdefault(entType, dict())
    NERcounter[entType].setdefault(lemma, 0)
    NERcounter[entType][lemma] += 1
    
print(NERcounter)
print(txt)

{'PER': {'дева': 1}}
Когда ты вся впадаешь в ступор
 
 когда ты в ступор вся впадаешь
 
 когда ты вся сама и штопор
 
 и пробки горестный затор
 
 когда ты вся одновременна
 
 какой предмет тебя ни встретит
 
 он тут же просится в двойняшки
 
 твоей добытой весовой породе
 
 вызванной наружу
 
 то с объяснением то с признаньем –
 
 стоишь автобусом закрытым
 
 или площадкой смотровой
 
 а справа жизнь идёт и слева
 
 ещё с утра кипела дева
 
 и в остальных стеклопакетах клубится дымчатый озон
 
 но всё меняется когда ты
 
 становишься
 
 невиновата
 
 и каменеющая снизу
 
 смотри же мне в последний раз
 
 
 Из книги «Сейчас пройдёт» (2021)
 
 #выбор_Леты_Югай


In [6]:
%%time

# NERcounter = dict()
# for rec in records:
#     lines = rec['Text'].split('\n')
#     for txt in lines:
#         doc = nlp(txt)
#         for ent in doc.ents:
#             lemma = ent.lemma_
#             entType = ent.label_
#             NERcounter.setdefault(entType, dict())
#             NERcounter[entType].setdefault(lemma, 0)
#             NERcounter[entType][lemma] += 1
            
# lst = []
# for key in NERcounter['LOC']:
#     lst.append((NERcounter['LOC'][key], key))
# sortedlst = sorted(lst, reverse=True)
# sortedlst[:20]

CPU times: user 36 µs, sys: 28 µs, total: 64 µs
Wall time: 143 µs
